# Propagate uncertainties with the `errors` add-on for CO2SYS-Matlab

<hr>
James Orr - 27 February 2018<br>

<img align="left" width="60%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png" \><br><br>

LSCE/IPSL, CEA-CNRS-UVSQ, Gif-sur-Yvette, France
<hr>

**Abstract**: This notebook shows how to use the new 'errors' add-on for CO2SYS-Matlab to propagate uncertainties. It uses CO2SYS-Matlab and the add-on routine `errors.m` (which itself calls another add-on routine `derivnum.m`) in `octave`, GNU's clone of Matlab. One can simply inspect the HTML version of this file or take advantage of being able to execute its commands interactively in your browser. The latter requires `jupyter notebook` and its `oct2py` add-on that includes the `octavemagic` interface for `octave`.

#### Table of Contents:
    1. Basics (install & load octave)
    2. Propagate errors: use of new `errors` add-on

## 1. Basics

#### Run interactively

If you are visualizing this after clicking on the link to this file on github, you are visualizing the HTML version of a jupyter notebook. Alternatively, you may run cells interactively and modify them if you have `jupyter notebook` installed on your machine.  To install that software, just download and the anaconda open software installer for your computing platform (Windows, OS X, or Linux) from https://www.continuum.io/downloads and then follow the easy install instructions at

https://docs.continuum.io/anaconda/install#

Then just download this `jupyter notebook` file as well as the 3 routines in the src directory (`CO2SYS.m`, `errors.m`, and `derivnum.m`).  Afterwards, you'll only need to install `octavemagic`, available in the `oct2py` package with the 1-line command in the following section.

#### Install `octavemagic`

#### To install the octavemagic funtionality, we must install `oct2py`, with the following command at the Unix prompt:

  `conda install -c conda-forge oct2py=3.6.5`
  
Then launch the notebook as usual with the following command:

  `jupyter notebook`


A new window or tab should appear in your browser, showing the files in the directory from where you launched the above command. Then just click on one of the `.ipynb` files, such as this one.

Once the notebook file appears in your browser, move to any of its cells with your mouse. Run a cell by clicking on it and hitting Ctrl-Return.  Alternatively, type Shift-Return to run a cell and then move to the next one. More information on all the interactive commankds is available in the Jupyter Notebook Quick Start Guide: http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html

At the top of the notebook, you'll see 8 Tabs (File, Edit, View, Insert, ...). Those tabls provide commands that will allow you to do whatever you like. Under the Help tab you'll find keyboard shortcuts for commands. Alternatively, a cheat sheet for short cuts to commands within `jupyter notebook` is available at https://zenodo.org/record/44973/files/ipynb-cheat-sheet.pdf . Or use the command palette after typing Ctrl-Shift-P.

#### Documentation for octavemagic

Details on using octavemagic are here: https://ipython.org/ipython-doc/2/config/extensions/octavemagic.html

#### Load octave magic function

Because octavemagic is now in conda's oct2py module, it is loaded with a slight modification to what is given on the above web page, i.e., now with the command below

In [1]:
%load_ext oct2py.ipython

#### Specify the directory where you have put the Matlab routines `CO2SYS.m`, `errors.m`, and `derivnum.m`.

In [2]:
%%octave
addpath ("~/Software/MATLAB/CO2SYS-MATLAB/src")

## 2. Propagate errors with new `errors` add-on for CO2SYS-Matlab

### 2.1 Specify input variables and choices

In [3]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 2300;        % ALK
    PAR2 = 2000;        % DIC
    PAR1TYPE = 1;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 2;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 2;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 15; % set for K1 & K2: (a) 10=Lueker et al. (2000); (b) 14=Millero (2010)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

# Input for CO2SYS:
# --------------------------
    # Input variables for error propagation:
    r = 0.0;            % Correlation between **uncertainties** in PAR1 and PAR2 (-1 < r < 1)
    ePAR1 = 2;          % uncertainty in PAR1 (same units as PAR1)
    ePAR2 = 2;          % uncertainty in PAR2 (same units as PAR2)
    eSAL = 0;           % uncertainty in Salinity
    eTEMP = 0;          % uncertainty in Temperature
    eSI = 4;            % uncertainty in Sit
    ePO4 = 0.1;         % uncertainty in Pt
    #Default uncertainties (pK units): epK0 , epK1, epK2, epKb, epKw, epKspA, epKspC
    epK =                [0.004, 0.015, 0.03, 0.01, 0.01,   0.02,  0.02];
    #Default uncertainty for Total boron (0.01, i.e., a 1% relative uncertainty)
    eBt = 0.01;


### 2.2 Propagate uncertainties neglecting uncertainties from equilibrium constants & total boron

In [4]:
%%octave
% With no errors from Ks
    epK = 0.0;
    eBt = 0.0;
    [e, ehead, enice] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4,...
                              ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...,
                              pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
#   Print results
#    e
#    ehead
#    enice

# Print (nicely formatted):
  printf("%s   %s %s %s %s %s %s %s %s \n", ehead{1:9});
  printf("%f %f  %f  %f  %f %f %f     %f     %f \n", e(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

0.077469 3.805834  3.792600  3.408018  1.848113 0.130039 0.044126     0.028534     3.883373 

### 2.3 Propagate uncertainties sequentially accounting for errors from constants & total boron 

#### Default uncertainties in equilibrium constants, but assume no uncertainty in total boron $B_\text{T}$

In [5]:
%%octave
% With std errors from constants except for Bt
    epK = [0.004, 0.015, 0.03, 0.01, 0.01, 0.02, 0.02];
    eBt = 0.0;
    [ek, ekhead, eknice] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

0.346528 17.387650 17.327190 5.868302 4.704035 0.586355  0.253289     0.163791     17.741904 

#### Default uncertainties in equilibrium constants & default uncertainty in total $B_\text{T}$ (1%)

In [6]:
%%octave
% With std errors from constants & Bt
    epK = [0.004, 0.015, 0.03, 0.01, 0.01, 0.02, 0.02];
    eBt = 0.01;
    [ekb, ekbhead] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

0.347402 17.424799 17.364210 5.898451 4.746630 0.587641  0.253742     0.164084     17.779810 

Same calculation, but with a simpler way to specify the defaults for epK and eBt

In [7]:
%%octave
% With std errors from constants & Bt
    epK = '';
    eBt = '';
    [ekb, ekbhead] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

0.347402 17.424799 17.364210 5.898451 4.746630 0.587641  0.253742     0.164084     17.779810 

### 2.4 Summarize effects of accouning for uncertainties in equil. constants & total boron

#### Fractional increase in propagated uncertainty due to accounting for uncertainties from equilibrium constants (default values)

In [8]:
%%octave
  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", e(1:9));
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9) ./ e(1:9));


u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

4.473133 4.568684 4.568684 1.721911 2.545317 4.509063  5.740138     5.740138     4.568684 

Conclusion: Accounting for uncertainties from the equilibrium constants increases propagated uncertainties by 1.7 to 5.7 times for the At-Ct pair with default uncertainties.


#### Fractional increase in propagated uncertainty due to accounting for uncertainty in total boron (default value)

In [9]:
%%octave
  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9) ./ ek(1:9));


u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

1.002524 1.002137 1.002137 1.005138 1.009055 1.002193  1.001787     1.001787     1.002137 

Conclusion: Accounting for uncertainty in total boron increases propagated absolute uncertainties by less than 1% when using the default uncertainty (eBt = 0.01) but by up to 3.5% when using the eBt=0.02, i.e., with the At-Ct pair.


### 2.5 Compute percent relative errors 

#### Compute CO2SYS variables (the reference)

In [10]:
%%octave
% With std errors from Ks
    [d, dhead, dnice] = CO2SYS (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

#### Reorder CO2SYS output in new array having same order as output array from `errors` (above, section 2.3) for later division (below)

In [11]:
%%octave
ALK     = d(:,1);    %01 - TAlk                 (umol/kgSW)
DIC     = d(:,2);    %02 - TCO2                 (umol/kgSW)
pH      = d(:,3);    %03 - pHin                 ()
pCO2    = d(:,4);    %04 - pCO2 input           (uatm)
fCO2    = d(:,5);    %05 - fCO2 input           (uatm)
HCO3    = d(:,6);    %06 - HCO3 input           (umol/kgSW)
CO3     = d(:,7);    %07 - CO3 input            (umol/kgSW)
CO2     = d(:,8);    %08 - CO2 input            (umol/kgSW)
Hfree   = d(:,13);   %13 - Hfree input          (umol/kgSW)
OmegaCa = d(:,15);   %15 - OmegaCa input        ()
OmegaAr = d(:,16);   %16 - OmegaAr input        ()
xCO2    = d(:,17);   %17 - xCO2 input           (ppm)

H = 10**(-pH) * 1e9; # Convert pH to H+ and then from mol/kg to nmol/kg
dar = [H, pCO2, fCO2, HCO3, CO3, CO2, OmegaCa, OmegaAr, xCO2];
%dar

#### Compute percent error

In [12]:
%%octave
  perr = 100* e(1:9) ./ dar;

# Show errors in percent of base value computed with CO2SYS
  printf("%s  %s   %s   %s  %s    %s   %s %s %s \n", ehead{1:9});
  printf("%4.2f  %6.2f     %7.2f    %8.2f    %7.2f     %7.2f    %7.2f      %7.2f      %7.2f \n", perr(1:9));

u(Hin)  u(pCO2in)   u(fCO2in)   u(HCO3in)  u(CO3in)    u(CO2in)   u(OmegaCAin) u(OmegaARin) u(xCO2in) 

1.07    1.25        1.25        0.19       0.90        1.25       0.90         0.90         1.25 

#### Compute absolute error of pH from relative error in H

An absolute change in pH is equivalent to a relative change in H+.  That is, it can be shown that for **small** changes in H+ (dH):

\begin{equation}
d\hbox{pH} = - \frac{1}{\hbox{ln}(10)}\frac{d\hbox{H}}{\hbox{H}}
\hbox{.}
\end{equation}

To get to this result, 

1) start with the basic definition: $\hbox{pH} = - \hbox{log}_{10} \, [\hbox{H}^+]$,

2) convert the logarithm base 10 to a natural log with $\hbox{log}_{10}(x) = \hbox{ln}(x) / \hbox{ln}(10)$, 

3) take the derivative of each side, and

4) plug in the definition of the derivative of a natural log, i.e.,  $d \hbox{ln}(x) = dx/x$.

##### Do the actual calculation

In [13]:
%%octave
# We drop the minus sign below because errors are positive by definition (1 sigma)
# Note that "(0.01 * perr)" is the fractional error in hydrogen ion concentration (i.e., dH/H)
  dpH = (0.01 * perr(1))/log(10);
  dpH

dpH =  0.0046528

### 2.6 Example of using `errors.m` routine with the pH-At pair

In [14]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 2300;        % ALK
    PAR2 = 8.1;         % pH
    PAR1TYPE = 1;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 3;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 18;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

# Input Parameters:
    pHSCALEIN = 1;      %% pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 10; %% set for K1 & K2: (a) 10=Lueker et al. (2000); (b) 14=Millero (2010)
    KSO4CONSTANTS = 1;  %% KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

# Input for CO2SYS:
# --------------------------
    # Input variables for error propagation:
    r = 0.0;            % Correlation between **uncertainties** in PAR1 and PAR2 (-1 < r < 1)
    ePAR1 = 2;          % uncertainty in PAR1 (same units as PAR1)
    ePAR2 = 0.01;          % uncertainty in PAR2 (same units as PAR2)
    eSAL = 0;           % uncertainty in Salinity
    eTEMP = 0;          % uncertainty in Temperature
    eSI = 4;            % uncertainty in Sit
    ePO4 = 0.1;         % uncertainty in Pt
    #Default uncertainties: epK0 , epK1, epK2, epKb, epKw, epKspA, epKspC
    epK =                [0.004, 0.015, 0.03, 0.01, 0.01,  0.02,  0.02];
    #Default uncertainty for Total Boron: 0.01 (i.e., a 1% relative error)
    eBt = 0.01;

In [15]:
%%octave
% With std errors from constants & Bt
    epK = '';
    eBt = '';
    [e, ehead, eunits] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s  %s %s %s %s \n", ehead{1:9});
  printf("%s   %s  %s    %s %s %s    %s      %s          %s \n", eunits{1:9});
  printf("%f  %f  %f  %f  %f %f  %f     %f     %f \n", e(1:9));

u(TCO2)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in)  u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in) 

(umol/kg)   (uatm)  (uatm)    (umol/kg) (umol/kg) (umol/kg)    ( )      ( )          (ppm) 

12.620052  16.147947  16.091798  23.607306  11.461813 0.540985  0.345069     0.223141     16.476945 